In [ ]:
from pathlib import Path
import pandas as pd
import glob
import numpy as np
import pyarrow.parquet
import pyarrow as pa
import seaborn as sns
#import tqdm
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import gc

import warnings
warnings.filterwarnings('ignore')

#### **読めそうなものから読んで見る**

sample_submission.csv

In [ ]:
pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')

train_labels.csv

In [ ]:
pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')

dcm ファイルを読めるように

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
dicom = pydicom.read_file('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-1.dcm')

In [ ]:
data = dicom.pixel_array
plt.imshow(data)

In [ ]:
list = glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/*')
list = sorted(list)

In [ ]:
fig, ax = plt.subplots(4, 6, figsize=(20, 20))

for i in range(24):
    dicom = pydicom.read_file(list[i])
    data = dicom.pixel_array
    plt.subplot(4,6,i+1)
    plt.imshow(data)

In [ ]:
!ls ../input/rsna-miccai-brain-tumor-radiogenomic-classification/train

In [ ]:
train = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
train

In [ ]:
test = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
test

In [ ]:
train['FLAIR'] = 0
train['T1w']=0
train['T1wCE']=0
train['T2w']=0

In [ ]:
test['FLAIR'] = 0
test['T1w']=0
test['T1wCE']=0
test['T2w']=0

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train["BraTS21ID"] = train["BraTS21ID"].apply(lambda x: str(x).zfill(5))
train

In [ ]:
test["BraTS21ID"] = test["BraTS21ID"].apply(lambda x: str(x).zfill(5))
test

In [ ]:
file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/FLAIR/*'.format(i)
len(glob.glob(file))

In [ ]:
a=0
for i in train['BraTS21ID']:
    file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/FLAIR/*'.format(i)
    train['FLAIR'][a] = len(glob.glob(file))
    file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T1w/*'.format(i)
    train['T1w'][a] = len(glob.glob(file))
    file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T1wCE/*'.format(i)
    train['T1wCE'][a] = len(glob.glob(file))
    file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T2w/*'.format(i)
    train['T2w'][a] = len(glob.glob(file))
    a+=1

In [ ]:
a=0
for i in test['BraTS21ID']:
    file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{}/FLAIR/*'.format(i)
    test['FLAIR'][a] = len(glob.glob(file))
    file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{}/T1w/*'.format(i)
    test['T1w'][a] = len(glob.glob(file))
    file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{}/T1wCE/*'.format(i)
    test['T1wCE'][a] = len(glob.glob(file))
    file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{}/T2w/*'.format(i)
    test['T2w'][a] = len(glob.glob(file))
    a+=1

In [ ]:
train

In [ ]:
test

簡単なテーブルが出来たのでこのデータをもとにlightGBMで分析してみる

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
#必要ライブラリのインポート
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# #データの読み込み
# train = pd.read_csv('train.tsv', delimiter = '\t') #tsvの場合は読み込む際にdelimiter = '\t'  Tab Separated Value でTSV
# test = pd.read_csv('test.tsv', delimiter = '\t')
submission = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')

data = pd.concat([train, test], sort = False)

train = data[:len(train)]
test = data[len(train):]

#データの整形
y_train = train['MGMT_value']
X_train = train.drop(['MGMT_value', 'BraTS21ID'], axis = 1) 
X_test = test.drop(['MGMT_value', 'BraTS21ID'], axis = 1) 

#LightGBMを用いて分析できるように交差検証の下準備
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.3, random_state = 0, stratify = y_train)

#LightGBMで分析できるようにデータ型を変更する
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train)

#パラメーターを設定する
params = {
    'objective' : 'binary'
}

#modelの学習を行う
model = lgb.train(params, lgb_train, 
                 valid_sets = [lgb_train, lgb_eval], 
                 verbose_eval = 10,
                 num_boost_round = 1000,
                 early_stopping_rounds = 10)

#学習したモデルで予測を行い、整形する
y_pred = model.predict(X_test, num_iteration = model.best_iteration)
y_pred = (y_pred > 0.5).astype(int)
print(y_pred)
print(submission.head())
submission['MGMT_value'] = y_pred
print(submission.head())
submission.to_csv('submission.csv', index = False)
# #予測・整形したデータを提出データに埋め込む
# submission[1] = y_pred
# # print(submission.head())
# submission.to_csv('submission_1.csv', index = False, header= False)

# # feature importanceを表示
# importance = pd.DataFrame(model.feature_importance(), index=X_train.columns, columns=['importance'])
# display(importance)

# #決定木の構造を可視化
# import matplotlib.pyplot as plt
# ax = lgb.plot_tree(model, tree_index=0, figsize=(20, 20), show_info=['split_gain'])
# plt.show()
# graph = lgb.create_tree_digraph(model, tree_index=0, format='png', name='Tree')
# graph.render(view=True)

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(20, 20))

file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/FLAIR/*'.format('01005')
file_path = sorted(glob.glob(file))

for i in range(23):

    dicom = pydicom.read_file(file_path[i])
    data = dicom.pixel_array
    plt.subplot(5,5,i+1)
    plt.title(i)
    plt.axis("off")
    plt.imshow(data)

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(20, 20))

file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/FLAIR/*'.format('01009')
file_path = sorted(glob.glob(file))

for i in range(23):

    dicom = pydicom.read_file(file_path[i])
    data = dicom.pixel_array
    plt.subplot(5,5,i+1)
    plt.title(i)
    plt.axis("off")
    plt.imshow(data)

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(20, 20))

file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T1w/*'.format('01005')
file_path = sorted(glob.glob(file))

for i in range(23):

    dicom = pydicom.read_file(file_path[i])
    data = dicom.pixel_array
    plt.subplot(5,5,i+1)
    plt.title(i)
    plt.axis("off")
    plt.imshow(data)

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(20, 20))

file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T1w/*'.format('01009')
file_path = sorted(glob.glob(file))

for i in range(23):

    dicom = pydicom.read_file(file_path[i])
    data = dicom.pixel_array
    plt.subplot(5,5,i+1)
    plt.title(i)
    plt.axis("off")
    plt.imshow(data)

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(20, 20))

file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T1wCE/*'.format('01005')
file_path = sorted(glob.glob(file))

for i in range(23):

    dicom = pydicom.read_file(file_path[i])
    data = dicom.pixel_array
    plt.subplot(5,5,i+1)
    plt.title(i)
    plt.axis("off")
    plt.imshow(data)

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(20, 20))

file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T1wCE/*'.format('01009')
file_path = sorted(glob.glob(file))

for i in range(23):

    dicom = pydicom.read_file(file_path[i])
    data = dicom.pixel_array
    plt.subplot(5,5,i+1)
    plt.title(i)
    plt.axis("off")
    plt.imshow(data)

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(20, 20))

file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T2w/*'.format('01005')
file_path = sorted(glob.glob(file))

for i in range(23):

    dicom = pydicom.read_file(file_path[i])
    data = dicom.pixel_array
    plt.subplot(5,5,i+1)
    plt.title(i)
    plt.axis("off")
    plt.imshow(data)

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(20, 20))

file ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T2w/*'.format('01009')
file_path = sorted(glob.glob(file))

for i in range(23):

    dicom = pydicom.read_file(file_path[i])
    data = dicom.pixel_array
    plt.subplot(5,5,i+1)
    plt.title(i)
    plt.axis("off")
    plt.imshow(data)

とりあえず簡単な予測してみる

In [ ]:
train.head()